In [18]:
# package imports
#basics
import os
import numpy as np
import pandas as pd
import re

#misc
import gc
import time
import warnings
from tqdm import tqdm

#viz
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#settings
start_time=time.time()
color = sns.color_palette()
sns.set_style("dark")
warnings.filterwarnings("ignore")

# plotly visulization
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# sklearn package
from collections import Counter
from scipy.misc import imread
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# gensim LDA approach
# https://www.kaggle.com/errearanhas/topic-modelling-lda-on-elon-tweets/notebook
# import plotly as py
from gensim import corpora, models, similarities

# https://www.kaggle.com/currie32/the-importance-of-cleaning-text
# text cleaning method #1
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import wordnet
stop_words = stopwords.words('english')

lem = WordNetLemmatizer()
tokenizer=ToktokTokenizer()

In [43]:
start_time=time.time()
#importing every dataset
business=pd.read_csv("./yelp_business.csv")
# business_attributes=pd.read_csv("./yelp_business_attributes.csv")
# business_hours=pd.read_csv("./yelp_business_hours.csv")
# check_in=pd.read_csv("./yelp_checkin.csv")
reviews=pd.read_csv("./yelp_review.csv")
# tip=pd.read_csv("./yelp_tip.csv")
# user=pd.read_csv("./yelp_user.csv")
end_time=time.time()
print("Took",end_time-start_time,"s")

Took 56.40050435066223 s


In [44]:
# take the reviewes from restaurants
business_idx = business['categories'].apply(lambda x: True if ('Restaurant' or 'restaurant') in x else False)
business = business.loc[business_idx]

In [47]:
# business.sort_values('review_count', ascending=False).review_count.sum()
# business = business.sort_values('review_count', ascending=False).head(1000).reset_index(drop=True)

In [48]:
reviews = reviews[['business_id', 'text', 'stars']].merge(business[['business_id', 'name']], on='business_id').dropna()
reviews.shape

(3221555, 4)

In [49]:
import gensim
import logging
import tempfile
from gensim import corpora, models, similarities

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "C:\Users\Dyn\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [50]:
#to seperate sentenses into words
def preprocess(comment):
    """
    Function to build tokenized texts from input comment
    """
    return gensim.utils.simple_preprocess(comment, deacc=True, min_len=3)

data_all=reviews['text'].apply(lambda x: preprocess(x))

In [51]:
print("After preprocessing:",data_all.iloc[30])

After preprocessing: ['saw', 'this', 'place', 'mind', 'chef', 'just', 'the', 'night', 'before', 'while', 'were', 'visiting', 'montreal', 'had', 'hit', 'they', 'serve', 'only', 'one', 'thing', 'but', 'they', 'serve', 'right', 'had', 'already', 'visited', 'the', 'infamous', 'schwart', 'days', 'prior', 'and', 'although', 'know', 'they', 'two', 'different', 'sandwiches', 'wilensky', 'sandwich', 'tasted', 'way', 'better', 'the', 'place', 'outdated', 'but', 'full', 'character', 'there', 'maybe', 'like', 'stools', 'the', 'bar', 'and', 'you', 'eat', 'there', 'close', 'and', 'personal', 'the', 'servers', 'who', 'are', 'wonderful', 'people', 'the', 'sandwich', 'looks', 'dinky', 'and', 'lame', 'but', 'the', 'bun', 'warm', 'unlike', 'schwart', 'and', 'when', 'you', 'bite', 'into', 'man', 'tasty', 'even', 'ordered', 'one', 'bring', 'back', 'vancouver', 'since', 'were', 'leaving', 'that', 'day', 'must', 'visit']


In [52]:
#Phrases help us group together bigrams :  new + york --> new_york
bigram = models.Phrases(data_all)

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()

C:\Users\Dyn\Anaconda3\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning:

ins

2018-04-29 11:13:03,109 : INFO : PROGRESS: at sentence #630000, processed 56274814 words and 5641018 word types
2018-04-29 11:13:05,120 : INFO : PROGRESS: at sentence #640000, processed 57134767 words and 5692781 word types
2018-04-29 11:13:07,121 : INFO : PROGRESS: at sentence #650000, processed 57997176 words and 5745126 word types
2018-04-29 11:13:09,373 : INFO : PROGRESS: at sentence #660000, processed 58930607 words and 5811752 word types
2018-04-29 11:13:11,930 : INFO : PROGRESS: at sentence #670000, processed 59889442 words and 5889257 word types
2018-04-29 11:13:14,272 : INFO : PROGRESS: at sentence #680000, processed 60861840 words and 5957780 word types
2018-04-29 11:13:16,677 : INFO : PROGRESS: at sentence #690000, processed 61870147 words and 6026671 word types
2018-04-29 11:13:18,718 : INFO : PROGRESS: at sentence #700000, processed 62721477 words and 6077854 word types
2018-04-29 11:13:20,748 : INFO : PROGRESS: at sentence #710000, processed 63568489 words and 6128610 wor

2018-04-29 11:15:56,291 : INFO : PROGRESS: at sentence #1350000, processed 120818612 words and 9293392 word types
2018-04-29 11:15:58,791 : INFO : PROGRESS: at sentence #1360000, processed 121653659 words and 9332573 word types
2018-04-29 11:16:01,352 : INFO : PROGRESS: at sentence #1370000, processed 122534066 words and 9376895 word types
2018-04-29 11:16:03,609 : INFO : PROGRESS: at sentence #1380000, processed 123425018 words and 9419001 word types
2018-04-29 11:16:05,963 : INFO : PROGRESS: at sentence #1390000, processed 124372627 words and 9464641 word types
2018-04-29 11:16:08,333 : INFO : PROGRESS: at sentence #1400000, processed 125313692 words and 9509594 word types
2018-04-29 11:16:10,673 : INFO : PROGRESS: at sentence #1410000, processed 126192467 words and 9551549 word types
2018-04-29 11:16:13,098 : INFO : PROGRESS: at sentence #1420000, processed 127040451 words and 9589508 word types
2018-04-29 11:16:15,187 : INFO : PROGRESS: at sentence #1430000, processed 127892021 wor

2018-04-29 11:18:53,083 : INFO : PROGRESS: at sentence #2060000, processed 181928558 words and 12202419 word types
2018-04-29 11:18:55,134 : INFO : PROGRESS: at sentence #2070000, processed 182745539 words and 12233933 word types
2018-04-29 11:18:57,231 : INFO : PROGRESS: at sentence #2080000, processed 183583588 words and 12269986 word types
2018-04-29 11:18:59,426 : INFO : PROGRESS: at sentence #2090000, processed 184434480 words and 12307217 word types
2018-04-29 11:19:01,585 : INFO : PROGRESS: at sentence #2100000, processed 185239383 words and 12345795 word types
2018-04-29 11:19:03,689 : INFO : PROGRESS: at sentence #2110000, processed 186094303 words and 12386749 word types
2018-04-29 11:19:05,838 : INFO : PROGRESS: at sentence #2120000, processed 186947106 words and 12423108 word types
2018-04-29 11:19:07,966 : INFO : PROGRESS: at sentence #2130000, processed 187802230 words and 12460730 word types
2018-04-29 11:19:10,079 : INFO : PROGRESS: at sentence #2140000, processed 18865

2018-04-29 11:21:23,859 : INFO : PROGRESS: at sentence #2770000, processed 241696790 words and 14931195 word types
2018-04-29 11:21:25,873 : INFO : PROGRESS: at sentence #2780000, processed 242513188 words and 14969714 word types
2018-04-29 11:21:27,871 : INFO : PROGRESS: at sentence #2790000, processed 243345793 words and 15005797 word types
2018-04-29 11:21:29,874 : INFO : PROGRESS: at sentence #2800000, processed 244158612 words and 15038979 word types
2018-04-29 11:21:32,093 : INFO : PROGRESS: at sentence #2810000, processed 245025017 words and 15087016 word types
2018-04-29 11:21:34,421 : INFO : PROGRESS: at sentence #2820000, processed 245861334 words and 15124765 word types
2018-04-29 11:21:36,671 : INFO : PROGRESS: at sentence #2830000, processed 246689151 words and 15163161 word types
2018-04-29 11:21:38,857 : INFO : PROGRESS: at sentence #2840000, processed 247528302 words and 15200639 word types
2018-04-29 11:21:41,003 : INFO : PROGRESS: at sentence #2850000, processed 24834

In [53]:
#check bigram collation functionality 
bigram[data_all.iloc[30]]

['saw',
 'this',
 'place',
 'mind',
 'chef',
 'just',
 'the',
 'night',
 'before',
 'while',
 'were',
 'visiting_montreal',
 'had',
 'hit',
 'they',
 'serve',
 'only',
 'one',
 'thing',
 'but',
 'they',
 'serve',
 'right',
 'had',
 'already',
 'visited',
 'the',
 'infamous',
 'schwart',
 'days',
 'prior',
 'and',
 'although',
 'know',
 'they',
 'two',
 'different',
 'sandwiches',
 'wilensky',
 'sandwich',
 'tasted',
 'way',
 'better',
 'the',
 'place',
 'outdated',
 'but',
 'full',
 'character',
 'there',
 'maybe',
 'like',
 'stools',
 'the',
 'bar',
 'and',
 'you',
 'eat',
 'there',
 'close',
 'and',
 'personal',
 'the',
 'servers',
 'who',
 'are',
 'wonderful',
 'people',
 'the',
 'sandwich',
 'looks',
 'dinky',
 'and',
 'lame',
 'but',
 'the',
 'bun',
 'warm',
 'unlike',
 'schwart',
 'and',
 'when',
 'you',
 'bite',
 'into',
 'man',
 'tasty',
 'even',
 'ordered',
 'one',
 'bring',
 'back',
 'vancouver',
 'since',
 'were',
 'leaving',
 'that',
 'day',
 'must',
 'visit']

In [54]:
def clean(word_list):
    """
    Function to clean the pre-processed word lists 
    
    Following transformations will be done
    1) Stop words removal from the nltk stopword list
    2) Bigram collation (Finding common bigrams and grouping them together using gensim.models.phrases)
    3) Lemmatization (Converting word to its root form : babies --> baby ; children --> child)
    """
    #remove stop words
    clean_words = [w for w in word_list if not w in stop_words]
    #collect bigrams
    clean_words = bigram[clean_words]
    #Lemmatize
    clean_words=[lem.lemmatize(word, "v") for word in clean_words]
    return(clean_words)   

In [55]:
#check clean function
print("Before clean:",data_all.iloc[1])
print("After clean:",clean(data_all.iloc[1]))

Before clean: ['the', 'shop', 'was', 'featured', 'mind', 'chef', 'with', 'anthony', 'bourdain', 'and', 'david', 'chang', 'momofuku', 'note', 'that', 'you', 'don', 'like', 'mustard', 'not', 'come', 'this', 'place', 'they', 'will', 'not', 'take', 'out', 'the', 'mustard', 'cut', 'the', 'sandwich', 'half', 'for', 'you', 'that', 'just', 'how', 'they', 'have', 'been', 'serving', 'since', 'this', 'more', 'grab', 'and', 'place', 'the', 'staff', 'behind', 'the', 'counter', 'were', 'friendly', 'and', 'happy', 'discuss', 'about', 'the', 'shop', 'and', 'its', 'history', 'there', 'aren', 'any', 'fancy', 'plates', 'utensils', 'here', 'all', 'the', 'sandwiches', 'were', 'served', 'napkin', 'brown', 'bag', 'wilensky', 'special', 'made', 'with', 'all', 'beef', 'bologna', 'and', 'salami', 'and', 'swiss', 'cheese', 'the', 'sandwich', 'was', 'glorious', 'simple', 'yet', 'good', 'you', 'don', 'know', 'what', 'kind', 'cheese', 'choose', 'would', 'recommend', 'the', 'swiss', 'cheese', 'over', 'the', 'cheddar

In [56]:
# %timeit clean(data_all.iloc[1])

In [57]:
#scale it to all text
start_clean = time.time()
data_all=data_all.apply(lambda x:clean(x))
end_clean=time.time()
print("Time till cleaning corpus:",end_clean-start_time,"s")

Time till cleaning corpus: 7787.420110464096 s


In [58]:
#create the dictionary
dictionary = corpora.Dictionary(data_all)
print("There are",len(dictionary),"number of words in the final dictionary")

2018-04-29 12:32:17,109 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-04-29 12:32:18,442 : INFO : adding document #10000 to Dictionary(26878 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:19,722 : INFO : adding document #20000 to Dictionary(38241 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:20,579 : INFO : adding document #30000 to Dictionary(46120 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:21,597 : INFO : adding document #40000 to Dictionary(54423 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:22,532 : INFO : adding document #50000 to Dictionary(59398 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:23,433 : INFO : adding document #60000 to Dictionary(64077 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:32:24

2018-04-29 12:33:14,318 : INFO : adding document #540000 to Dictionary(195219 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:15,077 : INFO : adding document #550000 to Dictionary(196268 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:16,015 : INFO : adding document #560000 to Dictionary(197558 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:16,961 : INFO : adding document #570000 to Dictionary(200500 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:17,971 : INFO : adding document #580000 to Dictionary(202473 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:18,876 : INFO : adding document #590000 to Dictionary(203759 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:33:19,683 : INFO : adding document #600000 to Dictionary(204918 unique tokens: [

2018-04-29 12:34:07,370 : INFO : adding document #1080000 to Dictionary(273477 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:08,296 : INFO : adding document #1090000 to Dictionary(274648 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:09,234 : INFO : adding document #1100000 to Dictionary(275841 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:10,112 : INFO : adding document #1110000 to Dictionary(276800 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:11,145 : INFO : adding document #1120000 to Dictionary(277875 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:12,052 : INFO : adding document #1130000 to Dictionary(279160 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:34:13,019 : INFO : adding document #1140000 to Dictionary(280411 unique to

2018-04-29 12:35:09,777 : INFO : adding document #1610000 to Dictionary(335843 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:12,695 : INFO : adding document #1620000 to Dictionary(336818 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:13,756 : INFO : adding document #1630000 to Dictionary(337607 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:14,832 : INFO : adding document #1640000 to Dictionary(338478 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:15,731 : INFO : adding document #1650000 to Dictionary(339364 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:17,018 : INFO : adding document #1660000 to Dictionary(340811 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:35:18,291 : INFO : adding document #1670000 to Dictionary(343004 unique to

2018-04-29 12:36:19,781 : INFO : adding document #2140000 to Dictionary(406117 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:22,070 : INFO : adding document #2150000 to Dictionary(406949 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:22,880 : INFO : adding document #2160000 to Dictionary(407653 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:23,868 : INFO : adding document #2170000 to Dictionary(408608 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:24,788 : INFO : adding document #2180000 to Dictionary(409210 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:25,754 : INFO : adding document #2190000 to Dictionary(410197 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:36:26,794 : INFO : adding document #2200000 to Dictionary(411323 unique to

2018-04-29 12:37:20,058 : INFO : adding document #2670000 to Dictionary(470577 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:21,318 : INFO : adding document #2680000 to Dictionary(471436 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:22,412 : INFO : adding document #2690000 to Dictionary(472526 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:24,180 : INFO : adding document #2700000 to Dictionary(473593 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:26,287 : INFO : adding document #2710000 to Dictionary(475310 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:28,359 : INFO : adding document #2720000 to Dictionary(476526 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:37:31,288 : INFO : adding document #2730000 to Dictionary(477500 unique to

2018-04-29 12:38:24,634 : INFO : adding document #3200000 to Dictionary(544383 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:38:25,584 : INFO : adding document #3210000 to Dictionary(546194 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:38:26,374 : INFO : adding document #3220000 to Dictionary(547965 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...)
2018-04-29 12:38:26,503 : INFO : built Dictionary(548401 unique tokens: ['amaze', 'around', 'bologna', 'friendly', 'helpful']...) from 3221555 documents (total 165025906 corpus positions)


There are 548401 number of words in the final dictionary


In [59]:
#convert into lookup tuples within the dictionary using doc2bow
print(dictionary.doc2bow(data_all.iloc[1]))
print("Wordlist from the sentence:",data_all.iloc[1])
#to check
print("Wordlist from the dictionary lookup:",
      dictionary[21],dictionary[22],dictionary[23],dictionary[24],dictionary[25],dictionary[26],dictionary[27])

[(2, 1), (3, 1), (5, 2), (7, 2), (8, 1), (9, 4), (10, 2), (11, 1), (12, 1), (13, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1)]
Wordlist from the sentence: ['shop', 'feature', 'mind', 'chef', 'anthony_bourdain', 'david_chang', 'momofuku', 'note', 'like', 'mustard', 'come', 'place', 'take', 'mustard', 'cut', 'sandwich', 'half', 'serve', 'since', 'grab', 'place', 'staff', 'behind', 'counter', 'friendly', 'happy', 'discuss', 'shop', 'history', 'fancy', 'plat', 'utensils', 'sandwich', 'serve', 'napkin', 'brown_bag', 'wilensky_special', 'make', 'beef', 'bologna', '

In [60]:
#scale it to all text
start_time=time.time()
corpus = [dictionary.doc2bow(text) for text in data_all]
end_corpus=time.time()
print("Time till corpus creation:",end_clean-start_time,"s")

Time till corpus creation: -370.7758812904358 s


In [61]:
start_time=time.time()
#create the LDA model
ldamodel = models.LdaMulticore(corpus=corpus, num_topics=15, id2word=dictionary, workers=4)
end_lda=time.time()
print("Time till LDA model creation:",end_lda-start_time,"s")

2018-04-29 13:06:32,727 : INFO : using symmetric alpha at 0.06666666666666667
2018-04-29 13:06:32,754 : INFO : using symmetric eta at 0.06666666666666667
2018-04-29 13:06:32,884 : INFO : using serial LDA version on this node
2018-04-29 13:07:17,602 : INFO : running online LDA training, 15 topics, 1 passes over the supplied corpus of 3221555 documents, updating every 8000 documents, evaluating every ~80000 documents, iterating 50x with a convergence threshold of 0.001000
2018-04-29 13:07:17,639 : INFO : training LDA model using 4 processes
2018-04-29 13:07:17,784 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/3221555, outstanding queue size 1
2018-04-29 13:07:33,923 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/3221555, outstanding queue size 2
2018-04-29 13:07:33,949 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/3221555, outstanding queue size 3
2018-04-29 13:07:33,970 : INFO : PROGRESS: pass 0, dispatched chunk 

2018-04-29 13:08:15,840 : INFO : topic #6 (0.067): 0.019*"food" + 0.015*"good" + 0.013*"make" + 0.013*"really" + 0.012*"great" + 0.011*"like" + 0.009*"place" + 0.007*"come" + 0.006*"service" + 0.006*"best"
2018-04-29 13:08:15,855 : INFO : topic #14 (0.067): 0.016*"great" + 0.013*"place" + 0.012*"food" + 0.011*"get" + 0.010*"good" + 0.010*"order" + 0.009*"like" + 0.009*"service" + 0.009*"make" + 0.006*"delicious"
2018-04-29 13:08:15,867 : INFO : topic #4 (0.067): 0.016*"order" + 0.014*"food" + 0.012*"get" + 0.012*"good" + 0.011*"place" + 0.010*"one" + 0.009*"come" + 0.009*"go" + 0.008*"restaurant" + 0.008*"time"
2018-04-29 13:08:15,909 : INFO : topic diff=0.188189, rho=0.277350
2018-04-29 13:08:15,922 : INFO : PROGRESS: pass 0, dispatched chunk #27 = documents up to #56000/3221555, outstanding queue size 12
2018-04-29 13:08:16,022 : INFO : PROGRESS: pass 0, dispatched chunk #28 = documents up to #58000/3221555, outstanding queue size 12
2018-04-29 13:08:18,370 : INFO : PROGRESS: pass 0,

2018-04-29 13:08:54,843 : INFO : topic #10 (0.067): 0.014*"fry" + 0.012*"chicken" + 0.011*"get" + 0.010*"good" + 0.009*"like" + 0.008*"come" + 0.008*"place" + 0.008*"sushi" + 0.008*"try" + 0.007*"order"
2018-04-29 13:08:54,857 : INFO : topic #1 (0.067): 0.019*"come" + 0.018*"order" + 0.017*"wait" + 0.014*"food" + 0.010*"back" + 0.010*"place" + 0.009*"time" + 0.009*"table" + 0.009*"get" + 0.009*"take"
2018-04-29 13:08:54,870 : INFO : topic #13 (0.067): 0.012*"order" + 0.012*"like" + 0.012*"get" + 0.009*"one" + 0.009*"back" + 0.008*"burger" + 0.007*"great" + 0.007*"food" + 0.007*"service" + 0.007*"come"
2018-04-29 13:08:54,884 : INFO : topic #8 (0.067): 0.015*"like" + 0.013*"get" + 0.011*"good" + 0.011*"food" + 0.010*"place" + 0.010*"would" + 0.009*"room" + 0.007*"make" + 0.006*"come" + 0.006*"really"
2018-04-29 13:08:54,898 : INFO : topic #12 (0.067): 0.020*"great" + 0.019*"place" + 0.019*"vegas" + 0.015*"food" + 0.011*"get" + 0.009*"come" + 0.009*"good" + 0.009*"wait" + 0.007*"would" +

2018-04-29 13:09:41,447 : INFO : topic #7 (0.067): 0.027*"good" + 0.015*"food" + 0.014*"place" + 0.013*"get" + 0.010*"pretty" + 0.009*"nice" + 0.008*"service" + 0.008*"order" + 0.007*"like" + 0.007*"really"
2018-04-29 13:09:41,460 : INFO : topic #6 (0.067): 0.018*"food" + 0.015*"good" + 0.014*"great" + 0.012*"make" + 0.011*"really" + 0.009*"like" + 0.008*"amaze" + 0.008*"place" + 0.007*"best" + 0.007*"service"
2018-04-29 13:09:41,477 : INFO : topic #4 (0.067): 0.016*"get" + 0.015*"order" + 0.015*"food" + 0.010*"come" + 0.010*"one" + 0.010*"time" + 0.010*"say" + 0.009*"place" + 0.009*"go" + 0.009*"good"
2018-04-29 13:09:41,492 : INFO : topic #9 (0.067): 0.021*"food" + 0.015*"good" + 0.015*"place" + 0.015*"great" + 0.015*"time" + 0.013*"love" + 0.013*"service" + 0.010*"best" + 0.008*"like" + 0.008*"come"
2018-04-29 13:09:42,435 : INFO : topic diff=0.087349, rho=0.133631
2018-04-29 13:09:42,448 : INFO : PROGRESS: pass 0, dispatched chunk #69 = documents up to #140000/3221555, outstanding 

2018-04-29 13:10:19,596 : INFO : topic #3 (0.067): 0.016*"place" + 0.011*"food" + 0.008*"back" + 0.008*"get" + 0.008*"come" + 0.008*"good" + 0.007*"time" + 0.007*"service" + 0.006*"would" + 0.006*"like"
2018-04-29 13:10:19,611 : INFO : topic #5 (0.067): 0.023*"food" + 0.017*"breakfast" + 0.014*"great" + 0.014*"get" + 0.012*"good" + 0.009*"come" + 0.009*"place" + 0.009*"portion" + 0.009*"service" + 0.008*"wait"
2018-04-29 13:10:19,628 : INFO : topic #10 (0.067): 0.013*"fry" + 0.012*"chicken" + 0.011*"good" + 0.010*"get" + 0.010*"like" + 0.009*"try" + 0.008*"order" + 0.008*"come" + 0.007*"sauce" + 0.007*"taste"
2018-04-29 13:10:19,642 : INFO : topic #6 (0.067): 0.017*"food" + 0.014*"great" + 0.014*"good" + 0.012*"make" + 0.010*"really" + 0.009*"like" + 0.008*"amaze" + 0.007*"place" + 0.007*"best" + 0.007*"service"
2018-04-29 13:10:19,657 : INFO : topic #8 (0.067): 0.017*"like" + 0.013*"get" + 0.010*"good" + 0.009*"food" + 0.009*"would" + 0.009*"place" + 0.007*"make" + 0.006*"room" + 0.00

...

2018-04-29 13:53:03,481 : INFO : merging changes from 8000 documents into a model of 3221555 documents
2018-04-29 13:53:08,780 : INFO : topic #1 (0.067): 0.038*"wait" + 0.037*"order" + 0.029*"get" + 0.024*"time" + 0.023*"table" + 0.023*"come" + 0.022*"take" + 0.022*"food" + 0.018*"minutes" + 0.014*"seat"
2018-04-29 13:53:08,794 : INFO : topic #13 (0.067): 0.064*"pizza" + 0.030*"burger" + 0.023*"fry" + 0.017*"good" + 0.015*"order" + 0.015*"cheese" + 0.015*"get" + 0.013*"like" + 0.011*"burgers" + 0.011*"place"
2018-04-29 13:53:08,814 : INFO : topic #0 (0.067): 0.022*"chicken" + 0.021*"good" + 0.015*"tacos" + 0.015*"wing" + 0.015*"chip" + 0.015*"get" + 0.015*"order" + 0.013*"food" + 0.012*"sauce" + 0.012*"burrito"
2018-04-29 13:53:08,829 : INFO : topic #11 (0.067): 0.024*"good" + 0.023*"food" + 0.016*"menu" + 0.014*"service" + 0.013*"place" + 0.012*"like" + 0.012*"would" + 0.012*"really" + 0.012*"restaurant" + 0.011*"nice"
2018-04-29 13:53:08,843 : INFO : topic #8 (0.067): 0.014*"die" + 0

2018-04-29 13:53:38,586 : INFO : topic #2 (0.067): 0.034*"sandwich" + 0.018*"salad" + 0.013*"like" + 0.012*"get" + 0.010*"good" + 0.010*"meat" + 0.010*"bread" + 0.009*"place" + 0.009*"try" + 0.009*"chicken"
2018-04-29 13:53:38,628 : INFO : topic diff=0.032310, rho=0.025433
2018-04-29 13:53:38,640 : INFO : PROGRESS: pass 0, dispatched chunk #1560 = documents up to #3122000/3221555, outstanding queue size 12
2018-04-29 13:53:39,257 : INFO : merging changes from 10000 documents into a model of 3221555 documents
2018-04-29 13:53:43,904 : INFO : topic #1 (0.067): 0.038*"wait" + 0.037*"order" + 0.029*"get" + 0.023*"time" + 0.023*"table" + 0.023*"come" + 0.022*"take" + 0.022*"food" + 0.018*"minutes" + 0.013*"seat"
2018-04-29 13:53:43,920 : INFO : topic #2 (0.067): 0.034*"sandwich" + 0.018*"salad" + 0.013*"like" + 0.012*"get" + 0.010*"good" + 0.010*"meat" + 0.010*"bread" + 0.009*"place" + 0.009*"try" + 0.009*"chicken"
2018-04-29 13:53:43,935 : INFO : topic #9 (0.067): 0.046*"great" + 0.045*"fo

2018-04-29 13:54:12,352 : INFO : topic #10 (0.067): 0.018*"sushi" + 0.017*"order" + 0.016*"chicken" + 0.014*"dish" + 0.014*"rice" + 0.014*"roll" + 0.012*"like" + 0.011*"sauce" + 0.011*"spicy" + 0.010*"taste"
2018-04-29 13:54:12,383 : INFO : topic diff=0.035667, rho=0.025238
2018-04-29 13:54:12,402 : INFO : PROGRESS: pass 0, dispatched chunk #1576 = documents up to #3154000/3221555, outstanding queue size 3
2018-04-29 13:54:13,289 : INFO : PROGRESS: pass 0, dispatched chunk #1577 = documents up to #3156000/3221555, outstanding queue size 2
2018-04-29 13:54:14,002 : INFO : PROGRESS: pass 0, dispatched chunk #1578 = documents up to #3158000/3221555, outstanding queue size 3
2018-04-29 13:54:14,018 : INFO : PROGRESS: pass 0, dispatched chunk #1579 = documents up to #3160000/3221555, outstanding queue size 4
2018-04-29 13:54:14,031 : INFO : PROGRESS: pass 0, dispatched chunk #1580 = documents up to #3162000/3221555, outstanding queue size 5
2018-04-29 13:54:14,048 : INFO : PROGRESS: pass 0,

2018-04-29 13:54:51,817 : INFO : topic #3 (0.067): 0.009*"one" + 0.008*"find" + 0.007*"make" + 0.006*"open" + 0.006*"location" + 0.006*"see" + 0.006*"place" + 0.006*"restaurant" + 0.005*"know" + 0.005*"walk"
2018-04-29 13:54:51,855 : INFO : topic #1 (0.067): 0.038*"wait" + 0.037*"order" + 0.029*"get" + 0.024*"time" + 0.022*"come" + 0.022*"take" + 0.022*"table" + 0.022*"food" + 0.019*"minutes" + 0.013*"seat"
2018-04-29 13:54:51,873 : INFO : topic #6 (0.067): 0.011*"dessert" + 0.011*"pasta" + 0.010*"delicious" + 0.010*"dish" + 0.009*"sauce" + 0.009*"dinner" + 0.008*"bread" + 0.008*"meal" + 0.008*"italian" + 0.007*"order"
2018-04-29 13:54:51,905 : INFO : topic #4 (0.067): 0.019*"order" + 0.017*"food" + 0.015*"get" + 0.014*"say" + 0.013*"ask" + 0.012*"time" + 0.011*"go" + 0.011*"service" + 0.010*"one" + 0.010*"come"
2018-04-29 13:54:51,959 : INFO : topic diff=0.035029, rho=0.025078
2018-04-29 13:54:51,969 : INFO : PROGRESS: pass 0, dispatched chunk #1604 = documents up to #3210000/3221555,

Time till LDA model creation: 2935.7776589393616 s


In [62]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [63]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\Dyn\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      14.355632        1       1 -0.066594 -0.142734
9      10.736101        1       2 -0.054851 -0.072949
11      9.903391        1       3 -0.064356 -0.080624
6       8.110622        1       4 -0.019644  0.097071
1       8.020701        1       5 -0.068964 -0.152493
3       7.890372        1       6 -0.027780 -0.161294
10      7.084177        1       7 -0.057869  0.128456
2       6.338472        1       8 -0.069873  0.083164
7       6.288261        1       9 -0.074975 -0.021123
12      5.862091        1      10 -0.041832 -0.135643
13      4.998284        1      11 -0.070337  0.056339
0       4.773589        1      12 -0.071263  0.115086
14      2.213763        1      13  0.031321  0.233839
5       2.117949        1      14  0.130545  0.123283
8       1.306595        1      15  0.526473 -0.070379, topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
6263   Default  5.485960e+05       pizza  5.485960e+05  30.0000  30.0000
188    Default  2.579697e+06        food  2.579697e+06  29.0000  29.0000
2359   Default  7.433990e+05     chicken  7.433990e+05  28.0000  28.0000
127    Default  5.750850e+05        wait  5.750850e+05  27.0000  27.0000
88     Default  1.423005e+06       great  1.423005e+06  26.0000  26.0000
9      Default  3.717070e+05    sandwich  3.717070e+05  25.0000  25.0000
56     Default  1.714437e+06       order  1.714437e+06  24.0000  24.0000
625    Default  4.225890e+05         bar  4.225890e+05  23.0000  23.0000
1416   Default  4.402460e+05         fry  4.402460e+05  22.0000  22.0000
38     Default  2.002609e+06        good  2.002609e+06  21.0000  21.0000
3155   Default  2.403030e+05      burger  2.403030e+05  20.0000  20.0000
78     Default  5.252070e+05       drink  5.252070e+05  19.0000  19.0000
1540   Default  1.743320e+05       steak  1.743320e+05  18.0000  18.0000
7      Default  2.156962e+06       place  2.156962e+06  17.0000  17.0000
373    Default  1.272890e+06     service  1.272890e+06  16.0000  16.0000
773    Default  2.875430e+05     minutes  2.875430e+05  15.0000  15.0000
104    Default  5.880930e+05       price  5.880930e+05  14.0000  14.0000
8355   Default  2.475340e+05       sushi  2.475340e+05  13.0000  13.0000
1107   Default  5.076480e+05       table  5.076480e+05  12.0000  12.0000
1536   Default  5.090040e+05       sauce  5.090040e+05  11.0000  11.0000
606    Default  1.412360e+05   breakfast  1.412360e+05  10.0000  10.0000
86     Default  1.735300e+06         get  1.735300e+06   9.0000   9.0000
130    Default  5.598940e+05      always  5.598940e+05   8.0000   8.0000
3      Default  5.606290e+05    friendly  5.606290e+05   7.0000   7.0000
6043   Default  2.502120e+05        rice  2.502120e+05   6.0000   6.0000
1489   Default  4.691080e+05        dish  4.691080e+05   5.0000   5.0000
1534   Default  3.478700e+05       salad  3.478700e+05   4.0000   4.0000
24     Default  3.134350e+05      cheese  3.134350e+05   3.0000   3.0000
1113   Default  1.158560e+05      coffee  1.158560e+05   2.0000   2.0000
122    Default  6.394920e+05        love  6.394920e+05   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
3336   Topic15  1.525666e+03       aussi  1.526663e+03   4.3371  -7.2537
1968   Topic15  1.511502e+03        prix  1.512491e+03   4.3371  -7.2630
1403   Topic15  1.508763e+03         peu  1.509752e+03   4.3371  -7.2648
2578   Topic15  1.450729e+03     est_pas  1.451721e+03   4.3371  -7.3040
975    Topic15  1.423971e+03        meme  1.424961e+03   4.3371  -7.3227
417    Topic15  1.398396e+03       bonne  1.399386e+03   4.3370  -7.3408
5195   Topic15  1.339710e+03        leur  1.340700e+03   4.3370  -7.3837
3114   Topic15  1.361184e+03    tres_bon  1.362193e+03   4.3370  -7.3678
1026   Topic15  1.263493e+03        vous  1.26

In [65]:
ldamodel.save("whole-bigram-LDA")

2018-04-29 15:42:26,452 : INFO : saving LdaState object under whole-bigram-LDA.state, separately None
2018-04-29 15:42:27,132 : INFO : saved whole-bigram-LDA.state
2018-04-29 15:42:27,987 : INFO : saving LdaMulticore object under whole-bigram-LDA, separately ['expElogbeta', 'sstats']
2018-04-29 15:42:27,991 : INFO : storing np array 'expElogbeta' to whole-bigram-LDA.expElogbeta.npy
2018-04-29 15:42:28,280 : INFO : not storing attribute state
2018-04-29 15:42:28,283 : INFO : not storing attribute id2word
2018-04-29 15:42:28,285 : INFO : not storing attribute dispatcher
2018-04-29 15:42:28,309 : INFO : saved whole-bigram-LDA


In [68]:
data_all.to_csv("cleaned_text.csv",index=False)

In [69]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [70]:
dictionary.save_as_text("whole-bigram-dictionary.txt")

2018-04-29 15:56:26,437 : INFO : saving dictionary mapping to whole-bigram-dictionary.txt


In [71]:
type(corpus)

list

In [73]:
myfile= open('whole-LDA-corpus.txt', 'w')
for item in corpus:
  myfile.write("%s\n" % item)

In [74]:
type(bigram)

gensim.models.phrases.Phrases

In [76]:
bigram.save("whole-bigram.txt")

2018-04-29 16:10:30,774 : INFO : saving Phrases object under whole-bigram.txt, separately None
2018-04-29 16:11:25,331 : INFO : saved whole-bigram.txt


In [77]:
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [94]:
# n_top_words = 10
# print("\nTopics in LDA model: ")
# print_top_words(ldamodel, dictionary, n_top_words)
topics_df = pd.DataFrame(ldamodel.show_topics(num_topics=15, num_words=10)).rename(columns={0:'topic_num', 1:'topic_terms'})
for item in topics_df.topic_terms:
    print(item)

0.022*"chicken" + 0.021*"good" + 0.016*"chip" + 0.015*"wing" + 0.015*"get" + 0.014*"order" + 0.014*"tacos" + 0.013*"food" + 0.013*"sauce" + 0.012*"fish"
0.038*"wait" + 0.037*"order" + 0.029*"get" + 0.024*"time" + 0.022*"take" + 0.022*"come" + 0.022*"table" + 0.022*"food" + 0.019*"minutes" + 0.013*"seat"
0.035*"sandwich" + 0.017*"salad" + 0.013*"like" + 0.012*"get" + 0.010*"good" + 0.010*"bread" + 0.010*"meat" + 0.009*"place" + 0.009*"try" + 0.009*"lunch"
0.009*"one" + 0.008*"find" + 0.008*"make" + 0.006*"location" + 0.006*"open" + 0.006*"see" + 0.006*"place" + 0.005*"restaurant" + 0.005*"know" + 0.005*"walk"
0.019*"order" + 0.017*"food" + 0.015*"get" + 0.014*"say" + 0.013*"ask" + 0.012*"time" + 0.011*"go" + 0.011*"service" + 0.010*"one" + 0.010*"come"
0.040*"breakfast" + 0.033*"coffee" + 0.025*"und" + 0.017*"egg" + 0.011*"brunch" + 0.008*"cafe" + 0.007*"bacon" + 0.006*"crepe" + 0.006*"pancakes" + 0.006*"good"
0.011*"pasta" + 0.011*"dessert" + 0.010*"delicious" + 0.010*"dish" + 0.009*"s

In [96]:
topic_name = ['Appetizers1', 'Services1', 'Appetizers2', 'Location', 'Services2', 'Breakfast', 'Italian1', 'General', 'Extreme1', 'Service3', 
             'Japanese', 'Service4', 'Bar', 'Italian2', 'American']
topic_united = ['Appetizers', 'Services', 'Appetizers', 'Location', 'Services', 'Breakfast', 'Italian', 'General', 'Extreme', 'Service', 
             'Japanese', 'Service', 'Bar', 'Italian', 'American']
topics_df['topic_name'] = topic_name
topics_df['topic_united'] = topic_united
topics_df.head()

,topic_num,topic_terms,topic_name,topic_united
0,0,"0.022*""chicken"" + 0.021*""good"" + 0.016*""chip"" ...",Appetizers1,Appetizers
1,1,"0.038*""wait"" + 0.037*""order"" + 0.029*""get"" + 0...",Services1,Services
2,2,"0.035*""sandwich"" + 0.017*""salad"" + 0.013*""like...",Appetizers2,Appetizers
3,3,"0.009*""one"" + 0.008*""find"" + 0.008*""make"" + 0....",Location,Location
4,4,"0.019*""order"" + 0.017*""food"" + 0.015*""get"" + 0...",Services2,Services


In [167]:
business[business['name']  == '"Bacchanal Buffet"']

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
119907,RESDUcs7fIiihp38-d6_6g,"""Bacchanal Buffet""",The Strip,"""3570 S Las Vegas Blvd""",Las Vegas,NV,89109,36.116113,-115.176222,4.0,7009,1,Sandwiches;Buffets;Restaurants;Breakfast & Bru...


In [238]:
BB = reviews[reviews.business_id == 'RESDUcs7fIiihp38-d6_6g'].reset_index()
BB.head()

,index,business_id,text,stars,name
0,268702,RESDUcs7fIiihp38-d6_6g,"Date & Time: Tuesday, 2\/24\/15 @ 2pm\n\nThe b...",3,"""Bacchanal Buffet"""
1,268703,RESDUcs7fIiihp38-d6_6g,I'm staying at Caesar's Palace for 4th of July...,5,"""Bacchanal Buffet"""
2,268704,RESDUcs7fIiihp38-d6_6g,Great buffet and beautiful food presentation. ...,3,"""Bacchanal Buffet"""
3,268705,RESDUcs7fIiihp38-d6_6g,"Compared to other buffets in Vegas, Bacchanal ...",4,"""Bacchanal Buffet"""
4,268706,RESDUcs7fIiihp38-d6_6g,"By far the best buffet in Vegas!! A must stop,...",4,"""Bacchanal Buffet"""


In [172]:
test = BB.text
test = test.apply(lambda x: preprocess(x))
test = test.apply(lambda x:clean(x))
test.shape

(7006,)

In [180]:
corpus_test = [dictionary.doc2bow(text) for text in test]

In [219]:
result = ldamodel.inference(corpus_test)

In [228]:
result = [result[0][i].argmax() for i in tqdm(range(test.shape[0]))]

100%|██████████| 7006/7006 [00:00<00:00, 700200.01it/s]


In [239]:
BB[['topic_terms', 'topic_name', 'topic_united']] = topics_df.iloc[result][['topic_terms', 'topic_name', 'topic_united']].reset_index(drop=True)
BB.drop('index', axis=1, inplace=True)
BB.head()

,business_id,text,stars,name,topic_terms,topic_name,topic_united
0,RESDUcs7fIiihp38-d6_6g,"Date & Time: Tuesday, 2\/24\/15 @ 2pm\n\nThe b...",3,"""Bacchanal Buffet""","0.047*""food"" + 0.035*""good"" + 0.027*""place"" + ...",General,General
1,RESDUcs7fIiihp38-d6_6g,I'm staying at Caesar's Palace for 4th of July...,5,"""Bacchanal Buffet""","0.011*""pasta"" + 0.011*""dessert"" + 0.010*""delic...",Italian1,Italian
2,RESDUcs7fIiihp38-d6_6g,Great buffet and beautiful food presentation. ...,3,"""Bacchanal Buffet""","0.047*""food"" + 0.035*""good"" + 0.027*""place"" + ...",General,General
3,RESDUcs7fIiihp38-d6_6g,"Compared to other buffets in Vegas, Bacchanal ...",4,"""Bacchanal Buffet""","0.047*""food"" + 0.035*""good"" + 0.027*""place"" + ...",General,General
4,RESDUcs7fIiihp38-d6_6g,"By far the best buffet in Vegas!! A must stop,...",4,"""Bacchanal Buffet""","0.047*""food"" + 0.035*""good"" + 0.027*""place"" + ...",General,General


In [302]:
np.random.seed(11)
BB[BB.stars == 5].sample(1).text.values[0], BB[BB.stars == 1].sample(1).text.values[0]

("Omg where do I start the flavors were exquisite dishes abundant presentation beautiful go here it's a must",
 'Well, this buffet is the most worst I have been tried. I highly recommend you not to go if you from the city near by ocean or the city cooks good foods. This is not the one for people who have high standards on foods.')

In [290]:
np.random.seed(11111)
general_good = BB[(BB.topic_united == 'General') & (BB.stars == 5)].sample(1)
general_bad = BB[(BB.topic_united == 'General') & (BB.stars == 1)].sample(1)
general_good.text.values[0], general_bad.text.values[0]

("So much high end food and delicious desserts. Its one of the best buffets I've had in Vegas, and i have had them all.",
 'The foods are suck. We expect a good quality food but it did not happen in this buffet. TAKE YOUR MONEY AND GO SOMEWHERE ELSE. If u want to eat crab with ice, then enjoy.')

In [286]:
np.random.seed(1111)
appetizers_good = BB[(BB.topic_united == 'Appetizers') & (BB.stars == 5)].sample(1)
appetizers_bad = BB[(BB.topic_united == 'Appetizers') & (BB.stars == 1)].sample(1)
appetizers_good.text.values[0], appetizers_bad.text.values[0]

('Took my daughter for breakfast and was overwhelmed.  First plate was fried chicken n waffles, shrimp and grits, steak and eggs, and yogurt with granola.  All good.  Then I got an omelette, which was cooked very well, and then slathered on a bunch of fresh guacamole and salsa.  Finally got some ice cream and cookies.  I missed about 80 percent of the stuff there, too.  Worthwhile for us because it was early and not crowded at all.',
 "If you are a vegetarian there's not lot of options for you. The tomato soup tasted horrible, hot n sour soup was good. The chef don't speak English so they can't tell you more about the food at display, you have to go with your instinct about what to eat and what not to eat. Don't taste random things if you have food allergy, menu doesn't specify ingredients.")

In [296]:
np.random.seed(2018)
services_good = BB[(BB.topic_united == 'Service') & (BB.stars == 5)].sample(1)
services_bad = BB[(BB.topic_united == 'Service') & (BB.stars == 1)].sample(1)
services_good.text.values[0], services_bad.text.values[0]

('Amazing variety of great tasting food! You are guaranteed to leave satisfied. Service is great too!',
 "Brought our friends while they visit us. We always brought friends here while they visit us. This price for this buffet is increasing but quality is getting worse and worse. It's not worth at all.")

In [291]:
BB.to_csv("Bacchanal_topics.csv", index=False)

In [ ]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)